## 🪄 Install `raga-testing-platform` library

In [ ]:
pip install raga-testing-platform==1.0.47

### Put your `ACCESS KEY & SECRET KEY`

In [20]:
# Define the access key, secret key, and host
ACCESS_KEY = "Paste your access key here"
SECRET_KEY = "Paste your secret key here"
HOST = "http://65.0.13.122:8080/"

# Define the configuration directory path
CONFIG_DIR = "/root/.raga" #COLAB
# CONFIG_DIR = "~/.raga" # LOCAL MACHINE

!mkdir $CONFIG_DIR
!touch $CONFIG_DIR/config
!echo -e "[default]\nraga_access_key_id=$ACCESS_KEY\nraga_secret_access_key=$SECRET_KEY\napi_host=$HOST" > $CONFIG_DIR/config
!cat $CONFIG_DIR/config

mkdir: /Users/manabroy/.raga: File exists
[default]
raga_access_key_id=nnXvot82D3idpraRtCjJ
raga_secret_access_key=P2doycL4WBZXLNARIs4bESxttzF3MHSC5K15Jrs9
api_host=http://65.0.13.122:8080/


### Import All raga lib from raga module

In [ ]:
from raga import *
import datetime

### 🪄 Set up a Run

In [ ]:
project_name = "Automobile"
run_name = f"Demo-Run-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"
test_session = TestSession(project_name=project_name,run_name=run_name)

### 👟 Run AB Labelled Test

In [ ]:
rules = ModelABTestRules() 
rules.add(metric="precision_diff_all", IoU=0.5, _class="ALL", threshold=0.5)

model_comparison_check = model_ab_test(test_session=test_session, 
                                       dataset_name="automobile_dataset",
                                       test_name="AB-test-labelled",
                                       modelA = "modelA", 
                                       modelB = "modelB" ,
                                       type = "labelled", 
                                       gt="GT",
                                       rules = rules,
                                       aggregation_level=["location", "vehicle_no", "channel"])
# add payload into test_session object
test_session.add(model_comparison_check)
#run added ab test model payload
test_session.run()

### 👟 Run AB Unlabelled Test

In [ ]:
rules = ModelABTestRules() 
rules.add(metric = "difference_all", IoU = 0.5, _class = "ALL", threshold = 0.5)

model_comparison_check = model_ab_test(test_session=test_session, 
                                       dataset_name="automobile_dataset",
                                       test_name="AB-test-unlabelled",
                                       modelA = "modelA", 
                                       modelB = "modelB" ,
                                       type = "unlabelled", 
                                       rules = rules,
                                       aggregation_level=["location", "vehicle_no", "channel"])

# add payload into test_session object
test_session.add(model_comparison_check)
#run added test
test_session.run()

### 👟 Run Image Drift Detection Test

In [ ]:
rules = DriftDetectionRules()
rules.add(type="anomaly_detection", dist_metric="Euclidian", _class="ALL", threshold=0.6)

edge_case_detection = data_drift_detection(test_session=test_session,
                                           test_name="Drift-detection-test",
                                           train_dataset_name="automobile_dataset_train",
                                           field_dataset_name="automobile_dataset_field",
                                           train_embed_col_name="ImageEmbedding",
                                           field_embed_col_name = "ImageEmbedding" ,
                                           level = "image",
                                           aggregation_level=["location", "vehicle_no", "channel",],
                                           rules = rules)

# add payload into test_session object
test_session.add(edge_case_detection)
#run added test
test_session.run()